<a href="https://colab.research.google.com/github/evaalonsoortiz/7t-spine-coil/blob/acdc_spine_7t_007/7t_spine_coil_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **7T ACDC Spine Coil**

This Colab notebook is a demo for 7T ACDC Spine Coil Evaluation

#Setup

To setup, we will first clone the 7t-spine-coil and spinalcordtoolbox repos and then instal the spinalcordtoolbox.


In [ ]:
! git clone https://github.com/evaalonsoortiz/7t-spine-coil
% cd 7t-spine-coil
! git checkout acdc_spine_7t_007

! pip install -r requirements.txt
% cd ../

In [ ]:
!git clone --depth 1 --branch 5.3.0 https://github.com/spinalcordtoolbox/spinalcordtoolbox

In [ ]:
# Go to directory and install SCT
%cd spinalcordtoolbox/
!yes | ./install_sct

In [ ]:
# Activate environment
import os
os.environ['PATH'] += ':/content/spinalcordtoolbox/bin'
os.environ['SCT_DIR'] = '/content/spinalcordtoolbox'

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
from plotly.subplots import make_subplots
import nibabel as nib
import sys
import os
import cv2
import re
import pandas as pd
from os.path import join

# Segment the spinal cord

In [ ]:
% cd ../7t-spine-coil
! sct_deepseg_sc -i t1_mp2rage_sag_p3_1mm/acdc_spine_7t_007_20210616_112817275_t1_mp2rage_sag_p3_1mm_20210616100903_6.nii -c t1 -o t1_mp2rage_sag_p3_1mm/spinal_seg.nii -qc t1_mp2rage_sag_p3_1mm/qc


In [ ]:
! sct_register_multimodal -i tfl_sag_2mmISO_384mm/acdc_spine_7t_007_20210616_112817275_tfl_sag_2mmISO_384mm_20210616100903_9.nii -d t1_mp2rage_sag_p3_1mm/acdc_spine_7t_007_20210616_112817275_t1_mp2rage_sag_p3_1mm_20210616100903_6.nii -dseg t1_mp2rage_sag_p3_1mm/spinal_seg.nii -o tfl_sag_2mmISO_384mm/ -param step=1,type=im,metric=cc,algo=slicereg,poly=2,smooth=1 -qc tfl_sag_2mmISO_384mm/qc

In [ ]:
! sct_apply_transfo -i t1_mp2rage_sag_p3_1mm/spinal_seg.nii -d tfl_sag_2mmISO_384mm/acdc_spine_7t_007_20210616_112817275_tfl_sag_2mmISO_384mm_20210616100903_9.nii -w tfl_sag_2mmISO_384mm/warp_acdc_spine_7t_007_20210616_112817275_t1_mp2rage_sag_p3_1mm_20210616100903_62acdc_spine_7t_007_20210616_112817275_tfl_sag_2mmISO_384mm_20210616100903_9.nii.gz -o tfl_sag_2mmISO_384mm/spinal_seg_reg_tfl.nii 

# Plot B1+ map with overlaid SC ROI

In [ ]:
img = nib.load('tfl_sag_2mmISO_384mm/acdc_spine_7t_007_20210616_112817275_tfl_sag_2mmISO_384mm_20210616100903_9.nii')
seg = nib.load('tfl_sag_2mmISO_384mm/spinal_seg_reg_tfl.nii')

img_data = img.get_fdata()
seg_data = seg.get_fdata()
seg_data = np.ma.masked_where(seg_data < 0.3, seg_data)

# Overlay the two images
fig, ax = plt.subplots()
plt.axis('off')
img1 = ax.imshow(np.rot90(img_data[:,:,28]/10), cmap=plt.cm.gray)
ax.imshow(np.rot90(seg_data[:,:,28]), cmap=plt.cm.YlOrRd, interpolation='none')
plt.colorbar(img1)
img1.set_clim(vmin=0, vmax=120)
plt.title('B1+ [degrees]')
plt.show()


# Extract B1+ Metrics

In [ ]:
! sct_extract_metric -i tfl_sag_2mmISO_384mm/acdc_spine_7t_007_20210616_112817275_tfl_sag_2mmISO_384mm_20210616100903_9.nii -f tfl_sag_2mmISO_384mm/spinal_seg_reg_tfl.nii -method wa


## Plot Metrics

In [ ]:
# load metrics
df = pd.read_csv("extract_metric.csv", sep=",")

#df = df[(df != 0).all(1)]
df = df[(df['Size [vox]'] > 5)]

df['WA()'] = pd.to_numeric(df['WA()'], errors='coerce')
df['WA()'] = df['WA()'].div(10)


# plot average signal in [nT/V]

# first compute voltage at socket
# Open the file for reading
with open('tfl_sag_2mmISO_384mm/acdc_spine_7t_007_20210616_112817275_tfl_sag_2mmISO_384mm_20210616100903_9.json') as fd:

    # Iterate over the lines
    for line in fd:

        # Capture one-or-more characters of non-whitespace after the initial match
        match = re.search(r'"TxRefAmp":(.*?),', line)

        # Did we find a match?
        if match:
            # Yes, process it
            refV = float(match.group(1))

SocketV = refV*10**-0.095  # [V]   
gamma = 42.58e6 # [Hz/T]

# plot average signal 
subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line 
fig = px.line(df['WA()'].mul((1/(180*gamma*0.001*SocketV))*10**9), y = 'WA()')
fig2 = px.line(df, y = 'WA()')

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="slice [I->S]"
subfig.layout.yaxis.title="B1+ [nT/V]"
subfig.layout.yaxis2.title="B1+ [degrees]"

subfig.show()

Investigating our missing factor 2 in $B_1$ scaling:
$\large FA (rad) = \int\limits_0^\tau \gamma B_1(t) t dt$


From Kyle's $B_1$ mapping code:
```
% Calculate the B1eff using a 1ms, pi-pulse at the acquisition voltage,
% then scale the efficiency by the ratio of the measured flip angle to the
% requested flip angle in the pulse sequence.
```
Assuming a 1ms $\pi$-pulse at the acquisition voltage (with constant $B_1(t)$):
$\large B_1 = \frac{\pi}{\gamma\times 10^{-3}} = \frac{\pi}{2\pi \times 42.58 \times 10^6 \times 10^{-3}} = \frac{1}{2 \times 42.58 \times 10^6 \times 10^{-3}}$


Scaling for any FA different from $\pi$:
$\large B_1 =  \frac{FA (rad)}{\pi} \times \frac{1}{2 \times 42.58 \times 10^6 \times 10^{-3}} = \frac{FA (rad)}{2\pi \times 42.58 \times 10^6 \times 10^{-3}}$


What's currently used in Kyle's code: 
```
% Convert the flip angle maps to B1+ efficiency maps [nT/V]
% Calculate the B1eff using a 1ms, pi-pulse at the acquisition voltage,
% then scale the efficiency by the ratio of the measured flip angle to the
% requested flip angle in the pulse sequence.
GAMMA = 2.675e8; % [rad / (s T)]
RequestedFA = 90; % saturation flip angle -- hard-coded in sequence
VoltageAtSocket = AcqVoltage .* 10^-0.095; % Voltage at the socket 
B1eff_mag = (AcquiredFA ./ RequestedFA) .* (pi ./ (GAMMA .* 1e-3 .* VoltageAtSocket)); % [T/V]
B1eff_mag = B1eff_mag .* 1e9; % [T/V] to [nT/V]
```
(Ignoring voltage scaling and conversion to nT for clarity)
$\large B_1 = \frac{FA(^{\circ})}{Requested FA (90^{\circ})} \times \frac{\pi}{2\pi \times 42.58 \times 10^6 \times 10^{-3}}$

Coversion to rad:

$\large B_1 = \frac{FA(rad)}{\frac{\pi}{2}} \times \frac{\pi}{2\pi \times 42.58 \times 10^6 \times 10^{-3}} = \frac{FA(rad)}{\pi \times 42.58 \times 10^6 \times 10^{-3}}$

A factor 1/2 is missing compared to our results.

# Plot SNR map with overlaid SC ROI

In [ ]:
img = nib.load('coilQA_sag_FH_384mm_REF720_1p5x1p5x3mm/acdc_spine_7t_006_20210512_145613749_coilQA_sag_FH_384mm_REF720_1p5x1p5x3mm_20210512132653_24.nii')

img_data = img.get_fdata()

fig = px.imshow(np.rot90(img_data[:,:,3,0]),color_continuous_scale='gray',zmin=0, zmax=200)
fig.update_layout(
    height=800, width = 800,
    title_text='SNR (1.5 x 1.5 x 3mm^3)'
)
fig.show()
